In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/MyDrive/myDetection/ShapeRecogition/N5/OnePose_Plus_Plus/2024/CO2/NewApproach/Quantum_Data'

/content/gdrive/MyDrive/myDetection/ShapeRecogition/N5/OnePose_Plus_Plus/2024/CO2/NewApproach/Quantum_Data


In [ ]:
# !pip install pennylane --upgrade
!pip install openpyxl
!pip install pennylane pennylane-lightning[gpu] custatevec-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 137.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pennylane as qml
from pennylane import numpy as np_pennylane
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
from pennylane.optimize import AdamOptimizer
@qml.qnode(dev)
def quantum_circuit(inputs, weights):
    # Feature encoding
    for i in range(n_qubits):
        qml.RX(np.pi * inputs[i], wires=i)

    # Variational layers
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RZ(weights[l, i, 0], wires=i)
            qml.RY(weights[l, i, 1], wires=i)
            qml.RZ(weights[l, i, 2], wires=i)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# 3. QNN Model (Algorithm 3)
class QNN:
    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # Quantum parameters
        self.q_weights = np.random.uniform(0, 2*np.pi, (n_layers, n_qubits, 3))

        # Classical parameters
        self.pre_weights = np.random.randn(n_qubits, n_qubits)
        self.pre_bias = np.random.randn(n_qubits)
        self.post_weights = np.random.randn(1, n_qubits)
        self.post_bias = np.random.randn(1)

        # Quantum device
        self.dev = qml.device("default.qubit", wires=n_qubits)

        # Quantum circuit
        @qml.qnode(self.dev, interface="autograd")
        def quantum_circuit(inputs, weights):
            # Encode input
            for i in range(n_qubits):
                qml.RY(inputs[i], wires=i)

            # Variational layers
            for layer in range(n_layers):
                for i in range(n_qubits):
                    qml.Rot(*weights[layer, i], wires=i)
                for i in range(n_qubits-1):
                    qml.CNOT(wires=[i, i+1])

            return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

        self.quantum_circuit = quantum_circuit

    def forward(self, x):
        # Quantum part
        q_out = self.quantum_circuit(x, self.q_weights)

        # Classical part (using pennylane.numpy)
        hidden = np.tanh(np.dot(self.pre_weights, q_out) + self.pre_bias)
        output = np.dot(self.post_weights, hidden) + self.post_bias
        return output[0]

def train_qnn(X_train, y_train, X_val, y_val, epochs=50, lr=0.005, patience=10):
    model = QNN(n_qubits=4, n_layers=2)  # Adjust dimensions as needed
    opt = AdamOptimizer(stepsize=lr)

    best_loss = float('inf')
    patience_counter = patience
    best_params = None

    def cost_fn(params, x, y):
        # Unpack all parameters
        (q_weights, pre_weights, pre_bias,
         post_weights, post_bias) = params

        # Quantum part
        q_out = model.quantum_circuit(x, q_weights)

        # Classical part
        hidden = np.tanh(np.dot(pre_weights, q_out) + pre_bias)
        pred = np.dot(post_weights, hidden) + post_bias
        return (pred[0] - y) ** 2

    for epoch in range(epochs):
        # Training
        train_loss = 0
        for x, y in zip(X_train, y_train):
            params = (model.q_weights, model.pre_weights, model.pre_bias,
                     model.post_weights, model.post_bias)

            params, loss = opt.step_and_cost(cost_fn, params, x=x, y=y)

            # Update model parameters
            (model.q_weights, model.pre_weights, model.pre_bias,
             model.post_weights, model.post_bias) = params

            train_loss += loss

        train_loss /= len(X_train)

        # Validation
        val_loss = 0
        for x, y in zip(X_val, y_val):
            pred = model.forward(x)
            val_loss += (pred - y) ** 2
        val_loss /= len(X_val)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_params = (model.q_weights.copy(), model.pre_weights.copy(),
                          model.pre_bias.copy(), model.post_weights.copy(),
                          model.post_bias.copy())
            patience_counter = patience
        else:
            patience_counter -= 1
            if patience_counter == 0:
                print(f"Early stopping at epoch {epoch}")
                break

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Restore best parameters
    if best_params is not None:
        (model.q_weights, model.pre_weights, model.pre_bias,
         model.post_weights, model.post_bias) = best_params

    return model

# 4. Model Evaluation (Algorithm 4)
def evaluate_model(model, X_test, y_test):
    y_pred = np.array([model.forward(x) for x in X_test])
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}")
    return mse, rmse, r2, mae, y_pred

# 5. Data Simulation for Visualization (Algorithm 5)
def simulate_data():
    C_0 = 15
    beta = 0.15
    gamma = 0.1
    alpha = 0.5
    T_0 = 100
    P_CO2 = 10.408  # Fixed for Experiment 1 based on your data

    time = np.arange(1, 4, 0.165829146)  # Based on your data range
    temp = np.arange(104, 117.5, 0.5)
    conc = np.arange(100, 310, 10)
    shear = np.arange(210, 263, 1)

    data_sim = []
    for t in time:
        for T in temp:
            for I in conc:
                for S in shear:
                    C = C_0 * np.exp(-beta * t) * (1 + gamma * P_CO2) * \
                        (1 + alpha * (T - T_0) / T_0) * (1 - I / (I + 100))
                    data_sim.append([t, T, I, S, C])

    return pd.DataFrame(data_sim, columns=['time_hrs', 'Temperature_C',
                                          'concentration_ppm', 'Shear_Pa', 'corrosion_mm_yr'])

# 6. Visualization (Figures 1-3)
def plot_results(sim_data, df_actual):
    # Figure 1: 3D Surface Plot (Time vs Temperature vs Corrosion Rate)
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf_data = sim_data[sim_data['concentration_ppm'] == 200]  # Match your data
    ax.plot_trisurf(surf_data['time_hrs'], surf_data['Temperature_C'],
                    surf_data['corrosion_mm_yr'], cmap='Blues', alpha=0.8)
    ax.scatter(df_actual['time_hrs'], df_actual['Temperature_C'],
               df_actual['corrosion_mm_yr'], color='red', label='Actual Data')
    ax.set_xlabel('Time (hrs)')
    ax.set_ylabel('Temperature (°C)')
    ax.set_zlabel('Corrosion Rate (mm/yr)')
    ax.set_title('3D Surface Plot of Corrosion Rate')
    ax.legend()
    plt.savefig('3d_time_temp.png')
    plt.close()

    # Figure 2: Logarithmic Scatter Plot (Inhibitor Concentration vs Corrosion Rate)
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                    data=sim_data[sim_data['concentration_ppm'] == 200],
                    label='Simulated Data (200 ppm)', color='blue', marker='s')
    sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                    data=df_actual, label='Actual Data (200 ppm)', color='red', marker='o')
    plt.xscale('log')
    plt.xlabel('Inhibitor Concentration (ppm)')
    plt.ylabel('Corrosion Rate (mm/yr)')
    plt.title('Logarithmic Scatter Plot of Corrosion Rate')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.savefig('log_concentration.png')
    plt.close()

    # Figure 3: Bar Plot (Shear Stress vs Average Corrosion Rate)
    shear_bins = pd.cut(sim_data['Shear_Pa'], bins=[210, 220, 230, 240, 250, 263],
                        labels=['210-220', '220-230', '230-240', '240-250', '250-263'])
    shear_avg = sim_data.groupby(shear_bins)['corrosion_mm_yr'].mean()
    plt.figure(figsize=(8, 6))
    shear_avg.plot(kind='bar', color='purple')
    plt.xlabel('Shear Stress (Pa)')
    plt.ylabel('Average Corrosion Rate (mm/yr)')
    plt.title('Average Corrosion Rate by Shear Stress')
    plt.grid(True, ls="--")
    plt.savefig('shear_bar.png')
    plt.close()

In [ ]:
# Replace with your actual data path
data_path = 'SequentialDataInhibitor_large.xlsx'

# Load actual data for visualization
df_actual = pd.read_excel(data_path, engine='openpyxl')
# df_actual = pd.read_excel(data_path)

# Preprocess data
X_train, X_val, X_test, y_train, y_val, y_test, preprocessor = preprocess_data(data_path)

# Train QNN
model = train_qnn(X_train, y_train, X_val, y_val, epochs=50, lr=0.005, patience=10)

# Evaluate model
mse, rmse, r2, mae, y_pred = evaluate_model(model, X_test, y_test)

# Simulate data for visualization
sim_data = simulate_data()

# Generate plots
plot_results(sim_data, df_actual)

print("Output plots saved: 3d_time_temp.png, log_concentration.png, shear_bar.png")
print("Predictions saved in y_pred")

/usr/local/lib/python3.11/dist-packages/pennylane/_grad.py:310: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Epoch 0: Train Loss = 51.0475, Val Loss = 67.8121
Early stopping at epoch 10
MSE: 46.1713, RMSE: 6.7949, R²: -1.7311, MAE: 5.1780
Output plots saved: 3d_time_temp.png, log_concentration.png, shear_bar.png
Predictions saved in y_pred


/tmp/ipython-input-42-4197100460.py:206: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  shear_avg = sim_data.groupby(shear_bins)['corrosion_mm_yr'].mean()


In [ ]:
y_val.shape

(30,)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pennylane as qml
from pennylane import numpy as np_pennylane
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time
import torch
from sklearn.svm import SVR

# Set random seed for reproducibility
np.random.seed(42)

# 1. Data Preprocessing (Algorithm 1)
# Preprocess data with outlier removal and log transformation
def preprocess_data(data_path):
    print("Loading data from Excel sheets...")
    df = pd.concat(pd.read_excel(data_path, sheet_name=None), ignore_index=True)

    # Remove outliers based on corrosion_mm_yr
    mean = df['corrosion_mm_yr'].mean()
    std = df['corrosion_mm_yr'].std()
    df = df[(df['corrosion_mm_yr'] >= mean - 3 * std) & (df['corrosion_mm_yr'] <= mean + 3 * std)]

    # Apply log transformation to corrosion_mm_yr
    df['corrosion_mm_yr'] = np.log1p(df['corrosion_mm_yr'])

    # Select features (add pH if numeric)
    feature_columns = ['concentration_ppm', 'time_hrs', 'Pressure_bar_CO2', 'Temperature_C', 'Shear_Pa', 'Brine_Ionic_Strength']
    if df['pH'].dtype in [np.float64, np.int64]:
        feature_columns.append('pH')

    X = df[feature_columns].values
    y = df['corrosion_mm_yr'].values
    df_actual = df.copy()

    # Standardize features
    preprocessor = StandardScaler()
    X = preprocessor.fit_transform(X)

    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.833, random_state=42)

    print(f"Loaded {len(df)} samples after removing invalid entries.")
    print(f"Preprocessing completed in {time.time() - start_time:.2f} seconds.")
    print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)} samples")
    return X_train, X_val, X_test, y_train, y_val, y_test, preprocessor, df_actual

# 2. Quantum Circuit Construction (Algorithm 2)
# n_qubits = 8
# n_layers = 3
# dev = qml.device('default.qubit', wires=n_qubits)
# 2. Quantum Circuit Construction (Algorithm 2)
# 2. Quantum Circuit Construction (Algorithm 2)
# 2. Quantum Circuit Construction (Algorithm 2)
# 2. Quantum Circuit Construction (Algorithm 2)
n_qubits = 6
n_layers = 3
try:
    dev = qml.device('lightning.gpu', wires=n_qubits)
    print("Using GPU device: lightning.gpu")
except Exception as e:
    print(f"GPU device not available: {e}. Falling back to CPU.")
    dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, weights):
    # Feature encoding
    for i in range(n_qubits):
        qml.RX(np.pi * inputs[i], wires=i)

    # Variational layers
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RZ(weights[l, i, 0], wires=i)
            qml.RY(weights[l, i, 1], wires=i)
            qml.RZ(weights[l, i, 2], wires=i)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# 3. QNN Model (Algorithm 3)
class QNN(torch.nn.Module):
    def __init__(self, n_qubits, n_layers, n_features, n_hidden=16, device='cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.device = device
        # Initialize parameters as torch tensors
        self.weights = torch.nn.Parameter(torch.randn(n_layers, n_qubits, 3, device=self.device))
        self.pre_weights = torch.nn.Parameter(torch.randn(n_hidden, n_qubits, device=self.device))
        self.pre_bias = torch.nn.Parameter(torch.zeros(n_hidden, device=self.device))
        self.post_weights = torch.nn.Parameter(torch.randn(1, n_hidden, device=self.device))
        self.post_bias = torch.nn.Parameter(torch.zeros(1, device=self.device))

    def forward(self, x):
        # Ensure batch dimension
        if x.ndim == 1:
            x = x.unsqueeze(0)
        # Ensure input matches n_qubits
        if x.shape[1] > self.n_qubits:
            x = x[:, :self.n_qubits]
        elif x.shape[1] < self.n_qubits:
            padding = torch.zeros(x.shape[0], self.n_qubits - x.shape[1], device=self.device)
            x = torch.cat([x, padding], dim=1)
        # Process batch
        q_out = torch.empty(x.shape[0], self.n_qubits, device=self.device)
        for i in range(x.shape[0]):
            q_out[i] = torch.stack(quantum_circuit(x[i], self.weights))
        # Classical layers
        hidden = torch.matmul(self.pre_weights, q_out.T).T + self.pre_bias
        hidden = torch.tanh(hidden)
        output = torch.matmul(self.post_weights, hidden.T).T + self.post_bias
        return output.squeeze(-1)

def train_qnn(X_train, y_train, X_val, y_val, epochs=50, lr=0.0005, patience=10, batch_size=512):
    print("Starting QNN training...")
    start_time = time.time()

    model = QNN(n_qubits, n_layers, n_features=X_train.shape[1])
    opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-8)
    best_loss = float('inf')
    patience_counter = patience
    best_params = None

    # Convert data to torch tensors upfront
    X_train_torch = torch.as_tensor(X_train, dtype=torch.float32, device=model.device)
    y_train_torch = torch.as_tensor(y_train, dtype=torch.float32, device=model.device)
    X_val_torch = torch.as_tensor(X_val, dtype=torch.float32, device=model.device)
    y_val_torch = torch.as_tensor(y_val, dtype=torch.float32, device=model.device)

    n_batches = len(X_train) // batch_size + (1 if len(X_train) % batch_size else 0)

    for epoch in range(epochs):
        # Shuffle data
        indices = torch.randperm(len(X_train))
        X_train_shuffled = X_train_torch[indices]
        y_train_shuffled = y_train_torch[indices]

        # Training
        train_loss = 0
        for batch in range(n_batches):
            batch_start_time = time.time()
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, len(X_train))
            X_batch = X_train_shuffled[start_idx:end_idx]
            y_batch = y_train_shuffled[start_idx:end_idx]

            def cost_fn():
                preds = model.forward(X_batch)
                return torch.mean((preds - y_batch) ** 2)

            opt.zero_grad()
            loss = cost_fn()
            loss.backward()
            opt.step()
            train_loss += loss.item()
            print(f"Epoch {epoch}, Batch {batch + 1}/{n_batches}: Loss = {loss.item():.4f}, Time = {time.time() - batch_start_time:.2f}s")

        train_loss /= n_batches

        # Validation
        val_loss = 0
        with torch.no_grad():
            preds = model.forward(X_val_torch)
            val_loss = torch.mean((preds - y_val_torch) ** 2).item()

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_params = {k: v.clone().detach() for k, v in model.state_dict().items()}
            patience_counter = patience
        else:
            patience_counter -= 1
            if patience_counter == 0:
                print(f"Early stopping at epoch {epoch}")
                break

        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Time = {time.time() - start_time:.2f}s")

    # Restore best parameters
    model.load_state_dict(best_params)
    print(f"Training completed in {time.time() - start_time:.2f} seconds.")
    return model

# Function to analyze data distribution
def analyze_data(data_path):
    print("Analyzing data distribution...")
    df = pd.concat(pd.read_excel(data_path, sheet_name=None), ignore_index=True)
    # Select only numeric columns for correlation
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print("Data summary:")
    print(df[numeric_cols].describe())
    print("\nCorrelation with corrosion_mm_yr:")
    print(df[numeric_cols].corr()['corrosion_mm_yr'].sort_values())
    print("\nMissing values:")
    print(df.isnull().sum())
    # Check for outliers in corrosion_mm_yr
    mean = df['corrosion_mm_yr'].mean()
    std = df['corrosion_mm_yr'].std()
    outliers = df[(df['corrosion_mm_yr'] > mean + 3 * std) | (df['corrosion_mm_yr'] < mean - 3 * std)]
    print(f"\nOutliers in corrosion_mm_yr (>{mean + 3 * std:.2f} or <{mean - 3 * std:.2f}):")
    print(f"Number of outliers: {len(outliers)}")
    return df

# Update evaluate_model for vectorized input
def evaluate_model(model, X_test, y_test):
    model.eval()
    X_test_torch = torch.tensor(X_test, dtype=torch.float32, device=model.device)
    y_test_torch = torch.tensor(y_test, dtype=torch.float32, device=model.device)
    with torch.no_grad():
        y_pred = model.forward(X_test_torch).cpu().numpy()
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"QNN Test Results: MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}")
    return mse, rmse, r2, mae, y_pred


# 5. Data Simulation for Visualization (Algorithm 5)
def simulate_data():
    C_0 = 15
    beta = 0.15
    gamma = 0.1
    alpha = 0.5
    T_0 = 100
    P_CO2 = 10.408  # Fixed for Experiment 1

    time = np.arange(1, 4, 0.165829146)
    temp = np.arange(104, 117.5, 0.5)
    conc = np.arange(100, 310, 10)
    shear = np.arange(210, 263, 1)

    data_sim = []
    for t in time:
        for T in temp:
            for I in conc:
                for S in shear:
                    C = C_0 * np.exp(-beta * t) * (1 + gamma * P_CO2) * \
                        (1 + alpha * (T - T_0) / T_0) * (1 - I / (I + 100))
                    data_sim.append([t, T, I, S, C])

    return pd.DataFrame(data_sim, columns=['time_hrs', 'Temperature_C',
                                          'concentration_ppm', 'Shear_Pa', 'corrosion_mm_yr'])

# 6. Visualization (Figures 1-3)
def plot_results(df_actual, sim_data=None):
    print("Generating plots...")
    # Subsample actual data for visualization to avoid overcrowding
    df_sample = df_actual.sample(n=min(1000, len(df_actual)), random_state=42)

    # Figure 1: 3D Surface Plot (Time vs Temperature vs Corrosion Rate)
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    if sim_data is not None:
        surf_data = sim_data[sim_data['concentration_ppm'] == 200]
        ax.plot_trisurf(surf_data['time_hrs'], surf_data['Temperature_C'],
                        surf_data['corrosion_mm_yr'], cmap='Blues', alpha=0.8, label='Simulated')
    ax.scatter(df_sample['time_hrs'], df_sample['Temperature_C'],
               df_sample['corrosion_mm_yr'], color='red', label='Actual Data')
    ax.set_xlabel('Time (hrs)')
    ax.set_ylabel('Temperature (°C)')
    ax.set_zlabel('Corrosion Rate (mm/yr)')
    ax.set_title('3D Surface Plot of Corrosion Rate')
    ax.legend()
    plt.savefig('3d_time_temp.png')
    plt.close()

    # Figure 2: Logarithmic Scatter Plot (Inhibitor Concentration vs Corrosion Rate)
    plt.figure(figsize=(8, 6))
    if sim_data is not None:
        sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                        data=sim_data[sim_data['concentration_ppm'] == 200],
                        label='Simulated Data (200 ppm)', color='blue', marker='s')
    sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                    data=df_sample, label='Actual Data', color='red', marker='o')
    plt.xscale('log')
    plt.xlabel('Inhibitor Concentration (ppm)')
    plt.ylabel('Corrosion Rate (mm/yr)')
    plt.title('Logarithmic Scatter Plot of Corrosion Rate')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.savefig('log_concentration.png')
    plt.close()

    # Figure 3: Bar Plot (Shear Stress vs Average Corrosion Rate)
    shear_bins = pd.cut(df_sample['Shear_Pa'], bins=[210, 220, 230, 240, 250, 263],
                        labels=['210-220', '220-230', '230-240', '240-250', '250-263'])
    shear_avg = df_sample.groupby(shear_bins)['corrosion_mm_yr'].mean()
    plt.figure(figsize=(8, 6))
    shear_avg.plot(kind='bar', color='purple')
    plt.xlabel('Shear Stress (Pa)')
    plt.ylabel('Average Corrosion Rate (mm/yr)')
    plt.title('Average Corrosion Rate by Shear Stress')
    plt.grid(True, ls="--")
    plt.savefig('shear_bar.png')
    plt.close()



Using GPU device: lightning.gpu


In [ ]:

# Replace with your actual Excel file path
data_path = 'SequentialDataInhibitor_large.xlsx'

start_time = time.time()


# Analyze data
df = analyze_data(data_path)

# Preprocess data
X_train, X_val, X_test, y_train, y_val, y_test, preprocessor, df_actual = preprocess_data(data_path)

# Option to run only SVR for quick baseline
run_svr_only = False  # Set to True to skip QNN and run only SVR
if run_svr_only:
    svr, y_pred_svr = train_svr(X_train, y_train, X_test, y_test)
    pd.DataFrame({
        'y_test': np.expm1(y_test),  # Reverse log transformation
        'y_pred_svr': np.expm1(y_pred_svr)
    }).to_csv('predictions_svr.csv', index=False)
else:
    # Train QNN
    model = train_qnn(X_train, y_train, X_val, y_val, batch_size=512)

    # Evaluate QNN
    mse, rmse, r2, mae, y_pred_qnn = evaluate_model(model, X_test, y_test)
    # Reverse log transformation for metrics
    y_test_orig = np.expm1(y_test)
    y_pred_qnn_orig = np.expm1(y_pred_qnn)
    mse_orig = mean_squared_error(y_test_orig, y_pred_qnn_orig)
    rmse_orig = np.sqrt(mse_orig)
    r2_orig = r2_score(y_test_orig, y_pred_qnn_orig)
    mae_orig = mean_absolute_error(y_test_orig, y_pred_qnn_orig)
    print(f"QNN Original Scale - MSE: {mse_orig:.4f}, RMSE: {rmse_orig:.4f}, R2: {r2_orig:.4f}, MAE: {mae_orig:.4f}")

    # Train and evaluate SVR
    svr, y_pred_svr = train_svr(X_train, y_train, X_test, y_test)
    y_pred_svr_orig = np.expm1(y_pred_svr)
    mse_svr_orig = mean_squared_error(y_test_orig, y_pred_svr_orig)
    rmse_svr_orig = np.sqrt(mse_svr_orig)
    r2_svr_orig = r2_score(y_test_orig, y_pred_svr_orig)
    mae_svr_orig = mean_absolute_error(y_test_orig, y_pred_svr_orig)
    print(f"SVR Original Scale - MSE: {mse_svr_orig:.4f}, RMSE: {rmse_svr_orig:.4f}, R2: {r2_svr_orig:.4f}, MAE: {mae_svr_orig:.4f}")

    # Save predictions to CSV
    pd.DataFrame({
        'y_test': y_test_orig,
        'y_pred_qnn': y_pred_qnn_orig,
        'y_pred_svr': y_pred_svr_orig
    }).to_csv('predictions.csv', index=False)

# Generate plots
plot_results(df_actual, sim_data=None)

print("Output plots saved: 3d_time_temp.png, log_concentration.png, shear_bar.png")
print("Predictions saved to predictions.csv")

Analyzing data distribution...
Data summary:
         Experiment       Replica  concentration_ppm      time_hrs  \
count  15400.000000  15400.000000       15400.000000  15400.000000   
mean      12.831169      2.714286         244.935065     19.980519   
std        6.042047      1.536255         179.185937     12.618280   
min        1.000000      1.000000          10.000000      1.000000   
25%        8.000000      1.000000         100.000000      9.618090   
50%       14.000000      2.000000         200.000000     18.286432   
75%       18.000000      4.000000         500.000000     29.080402   
max       22.000000      6.000000         500.000000     59.000000   

       time_hrs_original  Pressure_bar_CO2  Temperature_C      Shear_Pa  \
count       15400.000000      15400.000000   15400.000000  15400.000000   
mean           19.980519          6.699337     103.081152    157.629989   
std            12.618280          3.313848      14.046774     71.886944   
min             1.000000

/tmp/ipython-input-34-3643313008.py:301: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  shear_avg = df_sample.groupby(shear_bins)['corrosion_mm_yr'].mean()


In [ ]:
import torch
import time
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pennylane as qml
import matplotlib.pyplot as plt
def preprocess_data(data_path):
    print("Loading data from Excel sheets...")
    df = pd.concat(pd.read_excel(data_path, sheet_name=None), ignore_index=True)

    # Remove outliers based on corrosion_mm_yr
    mean = df['corrosion_mm_yr'].mean()
    std = df['corrosion_mm_yr'].std()
    df = df[(df['corrosion_mm_yr'] >= mean - 3 * std) & (df['corrosion_mm_yr'] <= mean + 3 * std)]

    # Apply log transformation to corrosion_mm_yr
    df['corrosion_mm_yr'] = np.log1p(df['corrosion_mm_yr'])

    # Select features (add pH if numeric)
    feature_columns = ['concentration_ppm', 'time_hrs', 'Pressure_bar_CO2', 'Temperature_C', 'Shear_Pa', 'Brine_Ionic_Strength']
    if df['pH'].dtype in [np.float64, np.int64]:
        feature_columns.append('pH')

    X = df[feature_columns].values
    y = df['corrosion_mm_yr'].values
    df_actual = df.copy()

    # Standardize features
    preprocessor = StandardScaler()
    X = preprocessor.fit_transform(X)

    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.833, random_state=42)

    print(f"Loaded {len(df)} samples after removing invalid entries.")
    print(f"Preprocessing completed in {time.time() - start_time:.2f} seconds.")
    print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)} samples")
    return X_train, X_val, X_test, y_train, y_val, y_test, preprocessor, df_actual
# 2. Quantum Circuit Construction (Algorithm 2)
n_qubits = 4  # Reduced for speed
n_layers = 2  # Reduced for speed
try:
    dev = qml.device('lightning.gpu', wires=n_qubits)
    print("Using GPU device: lightning.gpu")
except Exception as e:
    print(f"GPU device not available: {e}. Falling back to CPU.")
    dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, weights):
    # Feature encoding
    for i in range(n_qubits):
        qml.RX(np.pi * inputs[i], wires=i)

    # Variational layers
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RZ(weights[l, i, 0], wires=i)
            qml.RY(weights[l, i, 1], wires=i)
            qml.RZ(weights[l, i, 2], wires=i)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# 3. QNN Model (Algorithm 3)
class QNN(torch.nn.Module):
    def __init__(self, n_qubits, n_layers, n_features, n_hidden=16, device='cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.device = device
        # Initialize parameters as torch tensors
        self.weights = torch.nn.Parameter(torch.randn(n_layers, n_qubits, 3, device=self.device))
        self.pre_weights = torch.nn.Parameter(torch.randn(n_hidden, n_qubits, device=self.device))
        self.pre_bias = torch.nn.Parameter(torch.zeros(n_hidden, device=self.device))
        self.post_weights = torch.nn.Parameter(torch.randn(1, n_hidden, device=self.device))
        self.post_bias = torch.nn.Parameter(torch.zeros(1, device=self.device))

    def forward(self, x):
        # Ensure batch dimension
        if x.ndim == 1:
            x = x.unsqueeze(0)
        # Ensure input matches n_qubits
        if x.shape[1] > self.n_qubits:
            x = x[:, :self.n_qubits]
        elif x.shape[1] < self.n_qubits:
            padding = torch.zeros(x.shape[0], self.n_qubits - x.shape[1], device=self.device)
            x = torch.cat([x, padding], dim=1)
        # Process batch
        q_out = torch.stack([torch.stack(quantum_circuit(x[i], self.weights)) for i in range(x.shape[0])])
        # Classical layers
        hidden = torch.matmul(self.pre_weights, q_out.T).T + self.pre_bias
        hidden = torch.tanh(hidden)
        output = torch.matmul(self.post_weights, hidden.T).T + self.post_bias
        return output.squeeze(-1)

def train_qnn(X_train, y_train, X_val, y_val, epochs=50, lr=0.001, patience=10, batch_size=256):
    print("Starting QNN training...")
    start_time = time.time()

    model = QNN(n_qubits, n_layers, n_features=X_train.shape[1])
    opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-8)
    best_loss = float('inf')
    patience_counter = patience
    best_params = None

    # Convert data to torch tensors upfront
    X_train_torch = torch.as_tensor(X_train, dtype=torch.float32, device=model.device)
    y_train_torch = torch.as_tensor(y_train, dtype=torch.float32, device=model.device)
    X_val_torch = torch.as_tensor(X_val, dtype=torch.float32, device=model.device)
    y_val_torch = torch.as_tensor(y_val, dtype=torch.float32, device=model.device)

    n_batches = len(X_train) // batch_size + (1 if len(X_train) % batch_size else 0)

    for epoch in range(epochs):
        # Shuffle data
        indices = torch.randperm(len(X_train))
        X_train_shuffled = X_train_torch[indices]
        y_train_shuffled = y_train_torch[indices]

        # Training
        train_loss = 0
        for batch in range(n_batches):
            batch_start_time = time.time()
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, len(X_train))
            X_batch = X_train_shuffled[start_idx:end_idx]
            y_batch = y_train_shuffled[start_idx:end_idx]

            def cost_fn():
                preds = model.forward(X_batch)
                return torch.mean((preds - y_batch) ** 2)

            opt.zero_grad()
            loss = cost_fn()
            loss.backward()
            opt.step()
            train_loss += loss.item()
            print(f"Epoch {epoch}, Batch {batch + 1}/{n_batches}: Loss = {loss.item():.4f}, Time = {time.time() - batch_start_time:.2f}s")

        train_loss /= n_batches

        # Validation
        val_loss = 0
        with torch.no_grad():
            preds = model.forward(X_val_torch)
            val_loss = torch.mean((preds - y_val_torch) ** 2).item()

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_params = {k: v.clone().detach() for k, v in model.state_dict().items()}
            patience_counter = patience
        else:
            patience_counter -= 1
            if patience_counter == 0:
                print(f"Early stopping at epoch {epoch}")
                break

        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Time = {time.time() - start_time:.2f}s")

    # Restore best parameters
    model.load_state_dict(best_params)
    print(f"Training completed in {time.time() - start_time:.2f} seconds.")
    return model

# Plotting function for comparison
def plot_predictions(y_test, y_pred_qnn, y_pred_svr, filename='predictions_comparison.png'):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred_qnn, label='QNN Predictions', alpha=0.5)
    plt.scatter(y_test, y_pred_svr, label='SVR Predictions', alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.xlabel('True Corrosion Rate (mm/yr)')
    plt.ylabel('Predicted Corrosion Rate (mm/yr)')
    plt.title('QNN vs SVR Predictions')
    plt.legend()
    plt.savefig(filename)
    plt.close()


# Function to analyze data distribution
def analyze_data(data_path):
    print("Analyzing data distribution...")
    df = pd.concat(pd.read_excel(data_path, sheet_name=None), ignore_index=True)
    # Select only numeric columns for correlation
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print("Data summary:")
    print(df[numeric_cols].describe())
    print("\nCorrelation with corrosion_mm_yr:")
    print(df[numeric_cols].corr()['corrosion_mm_yr'].sort_values())
    print("\nMissing values:")
    print(df.isnull().sum())
    # Check for outliers in corrosion_mm_yr
    mean = df['corrosion_mm_yr'].mean()
    std = df['corrosion_mm_yr'].std()
    outliers = df[(df['corrosion_mm_yr'] > mean + 3 * std) | (df['corrosion_mm_yr'] < mean - 3 * std)]
    print(f"\nOutliers in corrosion_mm_yr (>{mean + 3 * std:.2f} or <{mean - 3 * std:.2f}):")
    print(f"Number of outliers: {len(outliers)}")
    return df

# Update evaluate_model for vectorized input
def evaluate_model(model, X_test, y_test):
    model.eval()
    X_test_torch = torch.tensor(X_test, dtype=torch.float32, device=model.device)
    y_test_torch = torch.tensor(y_test, dtype=torch.float32, device=model.device)
    with torch.no_grad():
        y_pred = model.forward(X_test_torch).cpu().numpy()
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"QNN Test Results: MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}")
    return mse, rmse, r2, mae, y_pred


# 5. Data Simulation for Visualization (Algorithm 5)
def simulate_data():
    C_0 = 15
    beta = 0.15
    gamma = 0.1
    alpha = 0.5
    T_0 = 100
    P_CO2 = 10.408  # Fixed for Experiment 1

    time = np.arange(1, 4, 0.165829146)
    temp = np.arange(104, 117.5, 0.5)
    conc = np.arange(100, 310, 10)
    shear = np.arange(210, 263, 1)

    data_sim = []
    for t in time:
        for T in temp:
            for I in conc:
                for S in shear:
                    C = C_0 * np.exp(-beta * t) * (1 + gamma * P_CO2) * \
                        (1 + alpha * (T - T_0) / T_0) * (1 - I / (I + 100))
                    data_sim.append([t, T, I, S, C])

    return pd.DataFrame(data_sim, columns=['time_hrs', 'Temperature_C',
                                          'concentration_ppm', 'Shear_Pa', 'corrosion_mm_yr'])

# 6. Visualization (Figures 1-3)
def plot_results(df_actual, sim_data=None):
    print("Generating plots...")
    # Subsample actual data for visualization to avoid overcrowding
    df_sample = df_actual.sample(n=min(1000, len(df_actual)), random_state=42)

    # Figure 1: 3D Surface Plot (Time vs Temperature vs Corrosion Rate)
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    if sim_data is not None:
        surf_data = sim_data[sim_data['concentration_ppm'] == 200]
        ax.plot_trisurf(surf_data['time_hrs'], surf_data['Temperature_C'],
                        surf_data['corrosion_mm_yr'], cmap='Blues', alpha=0.8, label='Simulated')
    ax.scatter(df_sample['time_hrs'], df_sample['Temperature_C'],
               df_sample['corrosion_mm_yr'], color='red', label='Actual Data')
    ax.set_xlabel('Time (hrs)')
    ax.set_ylabel('Temperature (°C)')
    ax.set_zlabel('Corrosion Rate (mm/yr)')
    ax.set_title('3D Surface Plot of Corrosion Rate')
    ax.legend()
    plt.savefig('3d_time_temp.png')
    plt.close()

    # Figure 2: Logarithmic Scatter Plot (Inhibitor Concentration vs Corrosion Rate)
    plt.figure(figsize=(8, 6))
    if sim_data is not None:
        sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                        data=sim_data[sim_data['concentration_ppm'] == 200],
                        label='Simulated Data (200 ppm)', color='blue', marker='s')
    sns.scatterplot(x='concentration_ppm', y='corrosion_mm_yr',
                    data=df_sample, label='Actual Data', color='red', marker='o')
    plt.xscale('log')
    plt.xlabel('Inhibitor Concentration (ppm)')
    plt.ylabel('Corrosion Rate (mm/yr)')
    plt.title('Logarithmic Scatter Plot of Corrosion Rate')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.savefig('log_concentration.png')
    plt.close()

    # Figure 3: Bar Plot (Shear Stress vs Average Corrosion Rate)
    shear_bins = pd.cut(df_sample['Shear_Pa'], bins=[210, 220, 230, 240, 250, 263],
                        labels=['210-220', '220-230', '230-240', '240-250', '250-263'])
    shear_avg = df_sample.groupby(shear_bins)['corrosion_mm_yr'].mean()
    plt.figure(figsize=(8, 6))
    shear_avg.plot(kind='bar', color='purple')
    plt.xlabel('Shear Stress (Pa)')
    plt.ylabel('Average Corrosion Rate (mm/yr)')
    plt.title('Average Corrosion Rate by Shear Stress')
    plt.grid(True, ls="--")
    plt.savefig('shear_bar.png')
    plt.close()

Using GPU device: lightning.gpu


In [ ]:
data_path = 'SingleDoesdataInhibitor_large.xlsx'

start_time = time.time()


# Analyze data
df = analyze_data(data_path)

# Preprocess data
X_train, X_val, X_test, y_train, y_val, y_test, preprocessor, df_actual = preprocess_data(data_path)

# Option to run only SVR for quick baseline
run_svr_only = False  # Set to True to skip QNN and run only SVR
if run_svr_only:
    svr, y_pred_svr = train_svr(X_train, y_train, X_test, y_test)
    pd.DataFrame({
        'y_test': np.expm1(y_test),  # Reverse log transformation
        'y_pred_svr': np.expm1(y_pred_svr)
    }).to_csv('predictions_svr.csv', index=False)
else:
    # Train QNN
    model = train_qnn(X_train, y_train, X_val, y_val, batch_size=512)

    # Evaluate QNN
    mse, rmse, r2, mae, y_pred_qnn = evaluate_model(model, X_test, y_test)
    # Reverse log transformation for metrics
    y_test_orig = np.expm1(y_test)
    y_pred_qnn_orig = np.expm1(y_pred_qnn)
    mse_orig = mean_squared_error(y_test_orig, y_pred_qnn_orig)
    rmse_orig = np.sqrt(mse_orig)
    r2_orig = r2_score(y_test_orig, y_pred_qnn_orig)
    mae_orig = mean_absolute_error(y_test_orig, y_pred_qnn_orig)
    print(f"QNN Original Scale - MSE: {mse_orig:.4f}, RMSE: {rmse_orig:.4f}, R2: {r2_orig:.4f}, MAE: {mae_orig:.4f}")

    # Train and evaluate SVR
    svr, y_pred_svr = train_svr(X_train, y_train, X_test, y_test)
    y_pred_svr_orig = np.expm1(y_pred_svr)
    mse_svr_orig = mean_squared_error(y_test_orig, y_pred_svr_orig)
    rmse_svr_orig = np.sqrt(mse_svr_orig)
    r2_svr_orig = r2_score(y_test_orig, y_pred_svr_orig)
    mae_svr_orig = mean_absolute_error(y_test_orig, y_pred_svr_orig)
    print(f"SVR Original Scale - MSE: {mse_svr_orig:.4f}, RMSE: {rmse_svr_orig:.4f}, R2: {r2_svr_orig:.4f}, MAE: {mae_svr_orig:.4f}")

    # Save predictions to CSV
    pd.DataFrame({
        'y_test': y_test_orig,
        'y_pred_qnn': y_pred_qnn_orig,
        'y_pred_svr': y_pred_svr_orig
    }).to_csv('predictions.csv', index=False)

# Generate plots
plot_results(df_actual, sim_data=None)

print("Output plots saved: 3d_time_temp.png, log_concentration.png, shear_bar.png")
print("Predictions saved to predictions.csv")

Analyzing data distribution...
Data summary:
         Experiment       Replica  concentration_ppm      time_hrs  \
count  20750.000000  20750.000000       20750.000000  20750.000000   
mean      11.253012      2.686747         215.301205     21.927711   
std        6.168164      1.480248         168.779510     13.633354   
min        1.000000      1.000000          10.000000      1.000000   
25%        6.000000      1.000000         100.000000     10.614458   
50%       12.000000      2.000000         200.000000     20.265060   
75%       16.000000      4.000000         300.000000     31.722892   
max       22.000000      6.000000         500.000000     59.000000   

       time_hrs_original  Pressure_bar_CO2  Temperature_C      Shear_Pa  \
count       20750.000000      20750.000000   20750.000000  20750.000000   
mean           21.927711          6.154896     105.315129    163.381931   
std            13.633354          3.210114      15.622062     66.288953   
min             1.000000